In [ ]:
from Restructure import *
from new_K_seismicity_rate3 import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from AVN import *
import glob
from datetime import datetime
import scipy.io as scp

# Density-based clusters:

In [ ]:
path_to_df0 = "/home/xianglo/Bureau/DATA_AMATRICE/DBSCAN_David/new_all_run_files_for_etas/catalog_all/"

####Raw data:
df0 = pd.read_csv(path_to_df0+'raw_data_modified_version.csv')
###Structure : t, x, y, z, m, x_km, y_km

###Clusters from dbscan:
v0_n50_d0_3 = scp.loadmat("dbscan_data/v0_d03km_n50_every1.mat")["clusters"].reshape(-1) ##au moins 50 séismes à < 200 m, v = 0km/d : 1 séisme sur 1 ou 5 comme vertex

###Restructure:
def restructure_dbscan(dbscan) :
    
    new_dbscan = []
    for i in range(0, len(dbscan)):
        new_dbscan.append(int(dbscan[i]))
        
    new_dbscan = np.reshape(new_dbscan, -1)
    
    return new_dbscan

v0_n50_d0_3 = restructure_dbscan(v0_n50_d0_3)

df0["v0_n50_d0_3"] = v0_n50_d0_3

###Take into account m0:
m0 = -5
df0 = df0[df0.m >= m0]

###Remove all rows with a magnitude equal to a nan value:
df0 = df0.dropna()

df0.index = np.arange(0, len(df0))

df0

In [ ]:
###Define AVN MS:
A, V, N, df_AVN = AVN(df0)

df_AVN

In [ ]:
###Define colors code for plots:
from matplotlib import cm

##Get colors : 
cmap_name1 = cm.get_cmap('tab20', 20)
cmap_name2 = cm.get_cmap('tab20b', 20)
cmap_name3 = cm.get_cmap('tab20c', 20)
cmap_name4 = cm.get_cmap('tab10', 10)
cmap_name5 = cm.get_cmap('Set3', 12)

cmap1 = cmap_name1(np.linspace(0, 1, 20))
cmap2 = cmap_name2(np.linspace(0, 1, 20))
cmap3 = cmap_name3(np.linspace(0, 1, 20))
cmap4 = cmap_name4(np.linspace(0, 1, 10))
cmap5 = cmap_name5(np.linspace(0, 1, 12))

cmap = np.vstack([cmap1, cmap2, cmap3, cmap4, cmap5, 
                  cmap1, cmap2, cmap3, cmap4, cmap5, 
                  cmap1, cmap2, cmap3, cmap4, cmap5,
                  cmap1, cmap2, cmap3, cmap4, cmap5,
                  cmap1, cmap2, cmap3, cmap4, cmap5])
print(len(cmap))

# Inversion ETAS

# Output files obtained:

In [ ]:
%%time 

###################CHECK SWARMS###################

#Organizing files:
path = "/home/xianglo/Bureau/DATA_AMATRICE/DBSCAN_David/new_all_run_files_for_etas/v0_d03_n50_every1_clusters/output_dbscan_v0_d03_n50_every1/"
path_coords = "/home/xianglo/Bureau/DATA_AMATRICE/DBSCAN_David/new_all_run_files_for_etas/v0_d03_n50_every1_clusters/dbscan_parameter_files_v0_d03_n50_every1/"

count = 0
count_step1 = 0
get_cluster_num = []

##for final table of swarms saved:
all_t1_swarms = []
all_t2_swarms = []
all_x1_swarms = []
all_x2_swarms = []
all_y1_swarms = []
all_y2_swarms = []
all_z1_swarms = []
all_z2_swarms = []

all_select_ratio_swarms = []

##Parameter:
ratio_factor = 10
time_factor = 5 ##d
min_cluster_size = 50 ##same as when running DBSCAN

for num_cluster_HDBSCAN in range(1, df0["v0_n50_d0_3"].max()+1) :
   
    df2 = np_to_pd_3(path+"dbscan_clust"+str(num_cluster_HDBSCAN)+".dat")

    ###We are interest in the cumulative seismic events first:
    df00 = df2[df2.ind == 1]
    df1 = np.cumsum(df00.ind)

    ###Compute seismicty rate: 
    dn = int(np.ceil(len(df00)*5/100)) ##smooth windown ##taking 5% of cluster's data

    ###Observed data: 
    dff = df1
    EQ1_rate = seismicity_rate2 (dn, df00, dff)

    ###The model data:
    dff = df00.lam
    EQ2_rate = seismicity_rate2 (dn, df00, dff)

    ##obs/mod:
    ratio = EQ1_rate/EQ2_rate

    arg = np.argwhere((ratio >= ratio_factor)&(ratio != np.inf))
    arg = arg.T[0]

    if len(arg) >= min_cluster_size:

        ##time data:
        tps = df00.t.values

        ##label of swarms:
        label = np.zeros(len(arg))

        ##diff in time for only data which ratio >= ratio_factor:
        dt = tps[arg[1:]] - tps[arg[:-1]]
        max_dt = np.max(dt)

        ##Find the beginning of the swarms:
        arg2 = np.argwhere(dt >= time_factor)        
        arg2 = arg2.T[0] +1
        label[arg2] += 1
        label[0] = 1

        ##Determine swarms:
        cumsum = np.cumsum(label)

        ##create a table:
        data = {"t":tps, "lam_mod":EQ2_rate, "lam_obs":EQ1_rate, "ratio":ratio, "label": np.zeros(len(ratio))-1}
        df_swarms = pd.DataFrame(data)

        ##put the right label of swarms:
        df_swarms.label.iloc[arg] = cumsum

        ##select swarms name:
        swarms_label = np.unique(df_swarms.label)[1:]

        for i in swarms_label:
            select_swarms = df_swarms[df_swarms.label == i]
            swarm_size = len(select_swarms)
            
            count_step1 +=1

            if (swarm_size >= min_cluster_size) == True:

                count += 1

                ###Get spatial coords:
                param_file_name = path_coords+"dbscan_cluster"+str(num_cluster_HDBSCAN)
                with open(param_file_name) as f:
                    lines = f.readlines()
                    lines_coords = lines[2].split()
                    x1, x2, y1, y2, z1, z2 = float(lines_coords[0]), float(lines_coords[1]), float(lines_coords[2]), float(lines_coords[3]) , float(lines_coords[4]), float(lines_coords[5]) 
                    ##x:lat(°), y:lon(°), z:z(km)

                all_x1_swarms.append(x1) ##deg 
                all_x2_swarms.append(x2) ##deg
                all_y1_swarms.append(y1) ##deg
                all_y2_swarms.append(y2) ##deg
                all_z1_swarms.append(z1) ##km
                all_z2_swarms.append(z2) ##km

                get_cluster_num.append(num_cluster_HDBSCAN)

                all_t1_swarms.append(select_swarms.t.min()) ##d
                all_t2_swarms.append(select_swarms.t.max()) ##d

                ##Plot ratio:
                plt.figure(figsize=(15,5))
                plt.plot(df00.t,ratio)
                plt.xlabel("Time (d)", fontsize=20)
                plt.ylabel("Ratio (obs/mod)", fontsize=20)
                plt.axhline(y=ratio_factor, linewidth=2, color='purple', ls = '--')
                for j in swarms_label:
                    each_swarms = df_swarms[df_swarms.label == j]
                    swarm_size = len(each_swarms)

                    if swarm_size >= min_cluster_size:
                        plt.axvline(each_swarms.t.min(), c=cmap[int(j)], linestyle='--')
                        plt.axvline(each_swarms.t.max(), c=cmap[int(j)], linestyle='--')
                        plt.xlim(each_swarms.t.min()-1, each_swarms.t.max()+1)
                        
                        ##get ratio data:
                        select_ind = df_swarms.t[(df_swarms.t >=each_swarms.t.min()) & (df_swarms.t <= each_swarms.t.max())]
                        select_ind1 = select_ind.index[0]
                        select_ind2 = select_ind.index[-1]
                        select_ratio = ratio[select_ind1:select_ind2]
                        all_select_ratio_swarms.append(select_ratio)
                        print(j, select_ind1, select_ind2)
                        
                plt.title('cluster n°'+str(num_cluster_HDBSCAN)+' (HDBSCAN), mP=2, v=0km/d, ratio_factor ='+str(ratio_factor)+", time_factor =" +str(time_factor)+"d, dn ="+str(dn), fontsize=15)
                plt.xticks(fontsize=15,)
                plt.yticks(fontsize=15,)
                plt.grid(True)
                #plt.savefig("/home/xianglo/Bureau/DATA_AMATRICE/DBSCAN_David/new_all_run_files_for_etas/v0_d03_n50_every1_clusters/ratio_for_clusters/dbscan"+str(num_cluster_HDBSCAN)+"_zoom.png")
                plt.show()

                ###Plot of swarms:
                plt.figure(figsize=(15,5))
                plt.plot(df_swarms.t, EQ1_rate, '-k', label = "observed data")
                plt.plot(df_swarms.t, EQ2_rate, c = "crimson", label = "predicted data")

                for j in swarms_label:
                    each_swarms = df_swarms[df_swarms.label == j]
                    swarm_size = len(each_swarms)

                    if swarm_size >= min_cluster_size:
                        
                        plt.axvline(each_swarms.t.min(), c=cmap[int(j)], linestyle='--')
                        plt.axvline(each_swarms.t.max(), c=cmap[int(j)], linestyle='--')
                        plt.xlim(each_swarms.t.min()-1, each_swarms.t.max()+1)
                        
                plt.title('cluster n°'+str(num_cluster_HDBSCAN)+' (HDBSCAN), mP=2, v=0km/d, ratio_factor ='+str(ratio_factor)+", time_factor =" +str(time_factor)+"d, dn ="+str(dn), fontsize=15)
                plt.xlabel("Time (d)", fontsize=20)
                plt.ylabel("Seismicity rate", fontsize=20)
                plt.yscale("log")
                plt.xticks(fontsize=15,)
                plt.yticks(fontsize=15,)
                plt.grid(True)
                plt.legend(fontsize=15)
                #plt.savefig("/home/xianglo/Bureau/DATA_AMATRICE/DBSCAN_David/new_all_run_files_for_etas/v0_d03_n50_every1_clusters/SR_for_clusters/dbscan"+str(num_cluster_HDBSCAN)+"_zoom.png")
                plt.show()

##create final table of swarms:a
data = {"t1":all_t1_swarms, "t2":all_t2_swarms,
       "x1":all_x1_swarms, "x2":all_x2_swarms,
       "y1":all_y1_swarms, "y2":all_y2_swarms,
       "z1":all_z1_swarms, "z2":all_z2_swarms,
       "num_EQ":get_cluster_num}

df_final_swarms = pd.DataFrame(data)

In [ ]:
len(all_select_ratio_swarms) ##y a les doublons/triplons == 10 d'où 51 au lieu des 41
#all_select_ratio_swarms

In [ ]:
all_mean = []
all_Q2 = []
all_Q3 = []
all_min = []
all_max = []

#for i in range(0, 4):
#
for i in range(0, len(all_select_ratio_swarms)):
    a = all_select_ratio_swarms[i]
    a_mean = np.mean(a)
    a_Q2 = np.percentile(a, 50)  ##median
    a_Q3 = np.percentile(a, 75)  
    a_min = np.min(a)
    a_max = np.max(a)
    all_mean.append(a_mean)
    all_Q2.append(a_Q2)
    all_Q3.append(a_Q3)
    all_min.append(a_min)
    all_max.append(a_max)

df_ratio = pd.DataFrame({"mean":all_mean, "Q2":all_Q2, "Q3":all_Q3, "min":all_min, "max":all_max})
df_ratio

In [ ]:
##On retire les 10 doublons/triplons:
df_ratio = df_ratio.drop_duplicates(subset=["mean", "Q2", "Q3", "min", "max"], keep='first', inplace=False, ignore_index=False)
print(len(df_ratio))
df_ratio.index = np.arange(0, len(df_ratio))

df_ratio

In [ ]:
##On crée un datframe qui prend aussi les valeurs de ratio:
df_final_swarms_ratio = pd.concat([df_final_swarms, df_ratio], axis=1)

# df_final_swarms_ratio = df_final_swarms_ratio.sort_values("t1")
# df_final_swarms_ratio.index = np.arange(1, len(df_final_swarms_ratio)+1)

# df_final_swarms_ratio.to_csv("/home/xianglo/Bureau/DATA_AMATRICE/DBSCAN_David/new_all_run_files_for_etas/v0_d03_n50_every1_clusters/df_final_swarms_of_v0_d03_n50_every1_with_ratio.csv", index = False)

df_final_swarms_ratio

In [ ]:
df_final_swarms = df_final_swarms.sort_values("t1")
df_final_swarms.index = np.arange(1, len(df_final_swarms)+1)

#df_final_swarms.to_csv("v0_d03_n50_every5_clusters/df_final_swarms_of_v0_d03_n50_every1.csv", index = False)

df_final_swarms